In [22]:
import json
from torch.utils.data import DataLoader, Dataset

In [16]:
path = "/home/phuonglh/vlp/dat/nli/XNLI-1.0/vi.tok.jsonl"

In [24]:
class VietnameseXNLI(Dataset):
  def __init__(self, jsonlPath):
    self.X = []
    self.y = []
    with open(jsonlPath) as f:
      for line in f:
        sample = json.loads(line)
        self.X.append((sample["sentence1_tokenized"], sample["sentence2_tokenized"]))
        self.y.append(sample["gold_label"])
  
  def __len__(self):
    return len(self.y)
  
  def __getitem__(self, i):
    return self.X[i], self.y[i]


In [25]:
dataset = VietnameseXNLI(path)

In [27]:
len(dataset)

7500

In [28]:
dataloader = DataLoader(dataset, batch_size=32)

In [30]:
len(dataloader)

235

In [32]:
next(iter(dataloader))

[[('Và anh_ấy nói , Mẹ , con đã về nhà .',
   'Và anh_ấy nói , Mẹ , con đã về nhà .',
   'Và anh_ấy nói , Mẹ , con đã về nhà .',
   'Tôi đã không biết mình đang hướng tới mục_đích gì , ngay cả việc báo_cáo cho một địa_chỉ cho trước ở Washington .',
   'Tôi đã không biết mình đang hướng tới mục_đích gì , ngay cả việc báo_cáo cho một địa_chỉ cho trước ở Washington .',
   'Tôi đã không biết mình đang hướng tới mục_đích gì , ngay cả việc báo_cáo cho một địa_chỉ cho trước ở Washington .',
   'Anh_ấy không cần phải bắt_đầu .',
   'Anh_ấy không cần phải bắt_đầu .',
   'Anh_ấy không cần phải bắt_đầu .',
   'Và tôi giống như ỔN , và đó là thế !',
   'Và tôi giống như ỔN , và đó là thế !',
   'Và tôi giống như ỔN , và đó là thế !',
   'Tôi chỉ ở nguyên đó , cố_gắng hình_dung ra vấn_đề .',
   'Tôi chỉ ở nguyên đó , cố_gắng hình_dung ra vấn_đề .',
   'Tôi chỉ ở nguyên đó , cố_gắng hình_dung ra vấn_đề .',
   'Và , ừ , họ thực_sự đã dừng lại chuyến thăm gia_đình bởi_vì họ chỉ , chỉ cần xác_định rằng